In [19]:
from collections import namedtuple
import os
os.chdir("../")


In [18]:
os.chdir(r"C:\Users\khush\Desktop\FSDS\codebase\deepCNNClassifier\research")

In [20]:
os.getcwd()

'C:\\Users\\khush\\Desktop\\FSDS\\codebase\\deepCNNClassifier'

In [12]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir","source_URL","local_data_file","unzip_dir"
])

In [13]:
from deepClassifier.constant import *
from deepClassifier.utils import read_yaml,create_directories

In [27]:
class ConfigurationManager:

    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
                                                    source_URL=config.source_URL,
                                                    local_data_file=config.local_data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [28]:
from urllib import request
from zipfile import ZipFile
import zipfile

class DataIngestion:

    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(url=self.config.source_URL,
            filename=self.config.local_data_file)
            
            
    def _get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self,zf:ZipFile,f:str,working_dir:str):
        target_file_path = os.path.join(working_dir,f)
        if not os.path.exists(target_file_path):
            zf.extract(f,working_dir)
        
        if os.path.getsize(target_file_path) == 0:
            os.remove(target_file_path)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file,mode="r") as zf :
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files=list_of_files)
            
            for f in updated_list_of_files:
                self._preprocess(zf,f,self.config.unzip_dir)
            #ZipFile.extractall(zf,path=self.config.unzip_dir)


In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-09-21 18:10:21,451 : INFO : common] : yaml file: config\config.yaml loaded successfully
[2022-09-21 18:10:21,457 : INFO : common] : yaml file: params.yaml loaded successfully
[2022-09-21 18:10:21,461 : INFO : common] : created directory at: artifacts
[2022-09-21 18:10:21,464 : INFO : common] : created directory at: artifacts/data_ingestion
